In [1]:
import numpy as np

In [2]:
list_SD36 = ['6x2pt_LSSTxSO_Y1_1sample','6x2pt_LSSTxSO_Y6_1sample'] # input datav prefix
list_COV36 = ['cov_LSSTxSO_Y1_1sample_6x2pt.npy','cov_LSSTxSO_Y6_1sample_6x2pt.npy']

list_outfiles = ['6x2pt_LSSTxSO_Y1_1sample','6x2pt_LSSTxSO_Y6_1sample','3x2pt_LSSTxSO_Y1_1sample','3x2pt_LSSTxSO_Y6_1sample'] # output datav filename

dir_datav = "datav/"
dir_bary = "datav/"
dir_COV = "cov/"
dir_LPC = "baryons/"

In [3]:
class gen_LPCi():
    def __init__(self,id_SD,outfile,id_include=None):
        '''
            id_SD = 0~1, int
        '''
        self.id_SD = id_SD
        self.id_include = id_include
        self.outfile = outfile
        self.list_bary_scenario = ['_eagle','_illustris','_TNG100','_mb2','_owls_AGN','_HzAGN']
        self.datav_dmo = self.load_datav(dir_datav,bary_scenario="_dmo",isCutted=1)
        print(self.id_SD)
        print(self.id_include)
        
        self.load_COV()
        print(self.id_SD)
        print(self.datav_dmo.shape, self.COV_cut.shape)
        
        self.cal_invL()
        self.Ndata = len(self.datav_dmo)
        self.Nscenario = len(self.list_bary_scenario)
        
        self.build_Ratio()
        self.build_Delta()
        self.build_DeltaChy()
        self.SVD(in_Delta=self.DeltaChy)
        self.write_LPC()
        
    
    def load_datav(self,dir_path,bary_scenario,isCutted=0):
        fname_datav = dir_path+list_SD36[self.id_SD]+bary_scenario
        print(fname_datav)
        datav = np.genfromtxt(fname_datav,skip_header=0,dtype="int,double",usecols=[0,1],names=["ind","obs"])["obs"]
        if self.id_include is None:
            self.id_include = np.arange(datav.size)
        datav = datav[self.id_include]
        
        self.Ndata_full  = len(datav)
        self.takeout_ID  = np.where(datav > 1e-20)[0]
        self.zero_ID     = np.where(datav < 1e-20)[0]
        
        if isCutted==1:
            return np.array(datav)[self.takeout_ID]
        else:
            return np.array(datav)
        
            
    def load_COV(self):
        
        self.fname_COV = dir_COV+list_COV36[self.id_SD]
        print(self.fname_COV)
        self.COV_full = np.load(self.fname_COV)
        self.COV_full = self.COV_full[self.id_include][:,self.id_include]
        
        self.COV_cut  = self.COV_full[self.takeout_ID][:,self.takeout_ID]
        
    def output_invCOV(self, outfile, sub_ind=None):
        
        self.invcov_masked = np.zeros((self.Ndata_full,self.Ndata_full))
        invcov_cut = np.linalg.inv(self.COV_cut)
        print(invcov_cut.shape)
        print(self.takeout_ID.size)
        for i in range(self.takeout_ID.size):
            (self.invcov_masked[self.takeout_ID[i]])[self.takeout_ID] = invcov_cut[i]
        if sub_ind is None:
            narr = np.arange(self.Ndata_full)
            col1 = np.repeat(narr, self.Ndata_full)
            col2 = np.tile(narr, self.Ndata_full)
            np.savetxt(outfile, np.c_[col1, col2, self.invcov_masked.flatten()], fmt='%d %d %le')
        else:
            Nsub = sub_ind.size
            narr = np.arange(Nsub)
            col1 = np.repeat(narr, Nsub)
            col2 = np.tile(narr, Nsub)
            np.savetxt(outfile, np.c_[col1, col2, (self.invcov_masked[sub_ind][:,sub_ind]).flatten()], fmt='%d %d %le')
        print('inv cov saved!')
    
    def cal_invL(self):
        self.L    = np.linalg.cholesky(self.COV_cut)
        self.invL =  np.linalg.inv(self.L)
        
    def build_Ratio(self):
        self.Ratio     = np.zeros((self.Ndata,self.Nscenario))
        
        for j in range(self.Nscenario):
        
            datav_bary = self.load_datav(dir_bary,bary_scenario=self.list_bary_scenario[j],isCutted=1)
            self.Ratio.T[j] = datav_bary/self.datav_dmo
            
    def build_Delta(self):
        DeltaT = self.Ratio.T*self.datav_dmo-self.datav_dmo
        self.Delta = DeltaT.T
    
    def build_DeltaChy(self):
        self.DeltaChy = np.dot(self.invL,self.Delta)
        
    def SVD(self,in_Delta):
        # self.U stores PC modes
        # PC1 = self.U.T[0]  ; PC2 = self.U.T[1]
        self.U, self.Sdig, VT = np.linalg.svd(in_Delta,full_matrices=True)
    
    def gen_datav_cosmolike_format(self,datav_cut):
        
        datav_full = np.zeros(self.Ndata_full)
        datav_full[self.takeout_ID] = datav_cut
        
        return datav_full

    
    def write_LPC(self):
        
        self.fname_out = dir_LPC+'LPC_'+self.outfile
        
        self.LPC1 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[0]))
        self.LPC2 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[1]))
        self.LPC3 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[2]))
        self.LPC4 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[3]))
        self.LPC5 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[4]))
        self.LPC6 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[5]))
        
        col_names='LPC1 LPC2 LPC3 LPC4 LPC5 LPC6'
        
        LPC_Matrix=np.column_stack((self.LPC1,self.LPC2,self.LPC3,self.LPC4,self.LPC5,self.LPC6))

        np.savetxt(self.fname_out,LPC_Matrix, fmt='%.8e %.8e %.8e %.8e %.8e %.8e')

       

In [4]:
PCA0 = gen_LPCi(0, list_outfiles[0])
PCA1 = gen_LPCi(1, list_outfiles[1])

datav/6x2pt_LSSTxSO_Y1_1sample_dmo
0
[   0    1    2 ... 1962 1963 1964]
cov/cov_LSSTxSO_Y1_1sample_6x2pt.npy
0
(1509,) (1509, 1509)
datav/6x2pt_LSSTxSO_Y1_1sample_eagle
datav/6x2pt_LSSTxSO_Y1_1sample_illustris
datav/6x2pt_LSSTxSO_Y1_1sample_TNG100
datav/6x2pt_LSSTxSO_Y1_1sample_mb2
datav/6x2pt_LSSTxSO_Y1_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y1_1sample_HzAGN
datav/6x2pt_LSSTxSO_Y6_1sample_dmo
1
[   0    1    2 ... 1962 1963 1964]
cov/cov_LSSTxSO_Y6_1sample_6x2pt.npy
1
(1526,) (1526, 1526)
datav/6x2pt_LSSTxSO_Y6_1sample_eagle
datav/6x2pt_LSSTxSO_Y6_1sample_illustris
datav/6x2pt_LSSTxSO_Y6_1sample_TNG100
datav/6x2pt_LSSTxSO_Y6_1sample_mb2
datav/6x2pt_LSSTxSO_Y6_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y6_1sample_HzAGN


In [5]:
PCA0.output_invCOV('invcov_Y1_6x2pt_1sample')

(1509, 1509)
1509
inv cov saved!


In [6]:
PCA0.output_invCOV('invcov_Y1_3x2pt_1sample', np.arange(1650))

(1509, 1509)
1509
inv cov saved!


In [7]:
PCA1.output_invCOV('invcov_Y6_6x2pt_1sample')
PCA1.output_invCOV('invcov_Y6_3x2pt_1sample', np.arange(1650))

(1526, 1526)
1526
inv cov saved!
(1526, 1526)
1526
inv cov saved!


In [8]:
def cal_Qexp(PCA, scenario):
    datav_bary = PCA.load_datav(dir_path=dir_datav, bary_scenario='_'+scenario, isCutted=1)
    diff = datav_bary - PCA.datav_dmo
    diff_chy = np.dot(PCA.invL, diff)
    Qexp = np.dot(PCA.U.T,diff_chy)
    return Qexp

In [9]:
baryons = ['dmo','eagle','illustris','TNG100','mb2','owls_AGN','HzAGN']
Qexps = [None]*7
for i in range(7):
    Qexps[i] = cal_Qexp(PCA0, scenario=baryons[i])
for i in range(7):
    print(baryons[i], (Qexps[i])[0:4])

datav/6x2pt_LSSTxSO_Y1_1sample_dmo
datav/6x2pt_LSSTxSO_Y1_1sample_eagle
datav/6x2pt_LSSTxSO_Y1_1sample_illustris
datav/6x2pt_LSSTxSO_Y1_1sample_TNG100
datav/6x2pt_LSSTxSO_Y1_1sample_mb2
datav/6x2pt_LSSTxSO_Y1_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y1_1sample_HzAGN
dmo [0. 0. 0. 0.]
eagle [ 3.45492424  1.61954536 -0.27654294 -0.39829574]
illustris [31.74947126 -1.87730123 -0.36551451 -0.05448423]
TNG100 [ 4.32644174  1.25277771 -0.43095506 -0.20975138]
mb2 [-0.69724883 -0.89633721  0.75857833 -0.38733681]
owls_AGN [23.32466028  1.56306397  0.64538779  0.11737676]
HzAGN [ 6.48728668  1.77345932 -0.01536912  0.15500485]


In [10]:
Qexps = [None]*7
for i in range(7):
    Qexps[i] = cal_Qexp(PCA1, scenario=baryons[i])
for i in range(7):
    print(baryons[i], (Qexps[i])[0:4])

datav/6x2pt_LSSTxSO_Y6_1sample_dmo
datav/6x2pt_LSSTxSO_Y6_1sample_eagle
datav/6x2pt_LSSTxSO_Y6_1sample_illustris
datav/6x2pt_LSSTxSO_Y6_1sample_TNG100
datav/6x2pt_LSSTxSO_Y6_1sample_mb2
datav/6x2pt_LSSTxSO_Y6_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y6_1sample_HzAGN
dmo [0. 0. 0. 0.]
eagle [ 6.72872268  3.05596105  0.61273142 -0.66113826]
illustris [55.59652682 -3.36520104  0.89362014 -0.11472586]
TNG100 [ 8.00907478  2.34256125  0.94827804 -0.3562199 ]
mb2 [-1.21210763 -1.53190527 -1.70610264 -0.68153676]
owls_AGN [42.86621659  2.51249425 -1.48308147  0.21563805]
HzAGN [1.24995259e+01 3.05700160e+00 8.49679549e-03 2.88833671e-01]


In [11]:
nsource = 10
nshear = nsource*(nsource+1)/2
ntheta = 15
nlens = 10

ndata3x2pt= 1965 - (nlens + nsource + 1)*ntheta
PCA3x2pt_0 = gen_LPCi(0, list_outfiles[2], np.arange(ndata3x2pt))

ndata3x2pt= 1965 - (nlens + nsource + 1)*ntheta
PCA3x2pt_1 = gen_LPCi(1, list_outfiles[3], np.arange(ndata3x2pt))

datav/6x2pt_LSSTxSO_Y1_1sample_dmo
0
[   0    1    2 ... 1647 1648 1649]
cov/cov_LSSTxSO_Y1_1sample_6x2pt.npy
0
(1254,) (1254, 1254)
datav/6x2pt_LSSTxSO_Y1_1sample_eagle
datav/6x2pt_LSSTxSO_Y1_1sample_illustris
datav/6x2pt_LSSTxSO_Y1_1sample_TNG100
datav/6x2pt_LSSTxSO_Y1_1sample_mb2
datav/6x2pt_LSSTxSO_Y1_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y1_1sample_HzAGN
datav/6x2pt_LSSTxSO_Y6_1sample_dmo
1
[   0    1    2 ... 1647 1648 1649]
cov/cov_LSSTxSO_Y6_1sample_6x2pt.npy
1
(1268,) (1268, 1268)
datav/6x2pt_LSSTxSO_Y6_1sample_eagle
datav/6x2pt_LSSTxSO_Y6_1sample_illustris
datav/6x2pt_LSSTxSO_Y6_1sample_TNG100
datav/6x2pt_LSSTxSO_Y6_1sample_mb2
datav/6x2pt_LSSTxSO_Y6_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y6_1sample_HzAGN


In [12]:
Qexps = [None]*7
for i in range(7):
    Qexps[i] = cal_Qexp(PCA3x2pt_0, scenario=baryons[i])
for i in range(7):
    print(baryons[i], (Qexps[i])[0:4])

datav/6x2pt_LSSTxSO_Y1_1sample_dmo
datav/6x2pt_LSSTxSO_Y1_1sample_eagle
datav/6x2pt_LSSTxSO_Y1_1sample_illustris
datav/6x2pt_LSSTxSO_Y1_1sample_TNG100
datav/6x2pt_LSSTxSO_Y1_1sample_mb2
datav/6x2pt_LSSTxSO_Y1_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y1_1sample_HzAGN
dmo [0. 0. 0. 0.]
eagle [ 3.44311399  1.61653836 -0.26681044 -0.39703148]
illustris [31.6048604  -1.87164008 -0.35419882 -0.05478476]
TNG100 [ 4.31239205  1.25101396 -0.42069394 -0.20862296]
mb2 [-0.70584326 -0.89521647  0.73909324 -0.38602067]
owls_AGN [23.19848312  1.55817017  0.62579454  0.11797593]
HzAGN [ 6.45474007  1.76816848 -0.01061853  0.1531929 ]


In [13]:
Qexps = [None]*7
for i in range(7):
    Qexps[i] = cal_Qexp(PCA3x2pt_1, scenario=baryons[i])
for i in range(7):
    print(baryons[i], (Qexps[i])[0:4])

datav/6x2pt_LSSTxSO_Y6_1sample_dmo
datav/6x2pt_LSSTxSO_Y6_1sample_eagle
datav/6x2pt_LSSTxSO_Y6_1sample_illustris
datav/6x2pt_LSSTxSO_Y6_1sample_TNG100
datav/6x2pt_LSSTxSO_Y6_1sample_mb2
datav/6x2pt_LSSTxSO_Y6_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y6_1sample_HzAGN
dmo [0. 0. 0. 0.]
eagle [ 6.70689949  3.055206    0.60091269 -0.66007851]
illustris [55.36755197 -3.3593082   0.88078396 -0.1148116 ]
TNG100 [ 7.98207898  2.34308124  0.93633131 -0.35501177]
mb2 [-1.21432231 -1.53506229 -1.68245939 -0.68055368]
owls_AGN [42.68238467  2.50503597 -1.46039344  0.21587401]
HzAGN [1.24503827e+01 3.05356741e+00 1.52953583e-03 2.87318110e-01]
